# 102Flowers Image Classifier

This is the main notebook for the project. See the associated report (WIP) for more information.

### Imports

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

### Hyperparameters

In [ ]:
# TODO: Set hyperparameters.
training_batch_size = 20
validation_batch_size = 20
test_batch_size = 20
epochs = 1000
learning_rate = 0.001
momentum = 0.9
crop_size = 128

### Device

In [ ]:
# Default to CPU
device = torch.device("cpu")

# Switch to GPU if available
if torch.cuda.is_available():
	print(f"Found {torch.cuda.device_count()} GPUs. Using cuda:0.")
	device = torch.device("cuda:0")
else:
	print("No GPUs found, using CPU.")

### Load Dataset

In [ ]:
training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

validation_data = datasets.Flowers102(
    root = "data",
    split = "val",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

testing_data = datasets.Flowers102(
    root = "data",
    split = "test",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

### DataLoaders

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=training_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=validation_batch_size, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=test_batch_size, shuffle=True)

## Model

In [ ]:
classifications = F.one_hot(torch.tensor([e for e in range(0,102)]), num_classes=102)
classifications, classifications.size()

In [ ]:
class F102Classifier(nn.Module):
    
	def __init__(self):
		super(F102Classifier, self).__init__()
		
		self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.conv4 = nn.Conv2d(24, 48, 3)
		self.conv5 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(384, 1024)
		self.fc2 = nn.Linear(1024, 512)
		self.fc3 = nn.Linear(512, 102)
		

	def forward(self, x):

		x = self.pool(F.relu(self.conv1(x)))
		x = self.pool(F.relu(self.conv2(x)))
		x = self.pool(F.relu(self.conv3(x)))
		x = self.pool(F.relu(self.conv4(x)))
		x = self.pool(F.relu(self.conv5(x)))
		#print("after conv5: ", x.size())
		x = torch.flatten(x)
		#print("after flatten: ", x.size())
		x = x.view(training_batch_size, -1)
		#print("after view: ", x.size())
		x = F.relu(self.fc1(x))
		#print("after fc1: ", x.size())
		x = F.relu(self.fc2(x))
		#print("after fc2: ", x.size())
		x = self.fc3(x)
		#print("after fc3: ", x.size())
		#print(x)
		#print(x.size())
		return x

net = F102Classifier()


### Loss Function & Optimiser

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimiser = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

## Train

In [ ]:
for epoch in range(epochs):
    
	running_loss = 0

	for i, data in enumerate(train_dataloader, 0):
		inputs, labels = data

		optimiser.zero_grad()

		outputs = net(inputs)
		loss = loss_function(outputs, labels)
		loss.backward()
		optimiser.step()

		running_loss += loss.item()
		print(loss.item())
		if i % 51 == 50:
			#print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 51:.10f}')
			running_loss = 0.0

print('Finished Training')

### Save Model

In [ ]:
save_path = "./models/classifier.pth"
torch.save(net.state_dict(), save_path)